In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
from time import time

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bethanys08/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
from nltk.corpus import stopwords

In [4]:
# reviews = pd.read_csv('../data/reviews.csv')
# beers = pd.read_csv('../data/beers.csv')

# # row in which value of 'Age' column is more than 30
# has_text_review_mask = reviews.apply(lambda x: True if len(x['text'].strip()) > 0 else False , axis=1)

# reviews_with_text = reviews[has_text_review_mask]
# reviews_with_beer_info = reviews_with_text.merge(beers, left_on='beer_id', right_on='id')
# reviews_with_beer_info.to_pickle('../data/beers_reviews.pkl')

In [5]:
def getSentiment(x):
    if x < 3.0:
        return 'negative'
    else:
        return 'positive'

In [6]:
# reviews_with_beer_info = pd.read_pickle('../data/beers_reviews.pkl')
# reviews_with_beer_info['sentiment'] = reviews_with_beer_info.overall.apply(getSentiment)
# reviews_with_beer_info.to_pickle('../data/beers_reviews_with_sentiment.pkl')

reviews_with_beer_info = pd.read_pickle('../data/beers_reviews_with_sentiment.pkl')

In [7]:
from tqdm import tqdm

In [8]:
# processed_features = []

# for sentence in tqdm(reviews_with_beer_info.text):
#     # Remove all the special characters
#     processed_feature = re.sub(r'\W', ' ', sentence)

#     # remove all single characters
#     processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)

#     # Remove single characters from the start
#     processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature) 

#     # Substituting multiple spaces with single space
#     processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)

#     # Removing prefixed 'b'
#     processed_feature = re.sub(r'^b\s+', '', processed_feature)

#     # Converting to Lowercase
#     processed_feature = processed_feature.lower()

#     processed_features.append(processed_feature)

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_large, X_small, y_large, y_small = train_test_split(reviews_with_beer_info.text, 
                                    reviews_with_beer_info['sentiment'], 
                                    test_size=0.05, 
                                    random_state=284)

In [11]:
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X_small, 
                                                    y_small, 
                                                    test_size=0.3, 
                                                    random_state=284)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [13]:
extra_words = ['beer', 'oz', 'ml', 'write', 'review']
all_stop_words = stopwords.words('english') + extra_words
print(all_stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [18]:
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words=all_stop_words)
X_train = vectorizer.fit_transform(X_train_raw)
X_test = vectorizer.transform(X_test_raw)

In [15]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [16]:
def fit_predict(clf):
    # train
    print(clf)
    t0 = time()
    clf.fit(X_train, y_train)
    train_time = time() - t0
    print("train time: %0.3fs" % train_time)
    
    t0 = time()
    pred = clf.predict(X_test)
    test_time = time() - t0
    print("test time:  %0.3fs" % test_time)
    
    print(confusion_matrix(y_test,pred))
    print(classification_report(y_test,pred))
    print(accuracy_score(y_test, pred))

# Multinomial Naive Bayes

In [21]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.1)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
proba = clf.predict_proba(X_test)

In [23]:
def sentiment_from_proba(x):
    if x >= 0.5:
        return 'positive'
    else:
        return 'negative'

In [20]:
proba

array([[0.00365222, 0.99634778],
       [0.01097526, 0.98902474],
       [0.00501109, 0.99498891],
       ...,
       [0.0127687 , 0.9872313 ],
       [0.02017788, 0.97982212],
       [0.00589393, 0.99410607]])

In [28]:
multiple_probas = [proba[:,1], proba[:,1]]

In [37]:
np.average(multiple_probas, axis=0, weights=[0.5, 0.5])

array([0.99634778, 0.98902474, 0.99498891, ..., 0.9872313 , 0.97982212,
       0.99410607])

In [29]:
multiple_probas

[array([0.99634778, 0.98902474, 0.99498891, ..., 0.9872313 , 0.97982212,
        0.99410607]),
 array([0.99634778, 0.98902474, 0.99498891, ..., 0.9872313 , 0.97982212,
        0.99410607])]

In [24]:
new_preds = [sentiment_from_proba(x) for x in proba[:,1]]

In [25]:
new_preds

['positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',

In [22]:
pred

array(['positive', 'positive', 'positive', ..., 'positive', 'positive',
       'positive'], dtype='<U8')

In [27]:
print(sum(new_preds != pred))

0


In [ ]:
from sklearn.naive_bayes import MultinomialNB
fit_predict(MultinomialNB(alpha=0.1))

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


# Linear SVC

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
fit_predict(LinearSVC(penalty='l1', dual=False, tol=1e-4))

# Passive aggressive

In [ ]:
from sklearn.linear_model import PassiveAggressiveClassifier


In [ ]:
fit_predict(PassiveAggressiveClassifier())